In [1]:
import time
from petitRADTRANS import Radtrans
import petitRADTRANS.retrieval_examples.emission.master_retrieval_model as rm
from petitRADTRANS import nat_cst as nc
import petitRADTRANS.rebin_give_width as rgw
from scipy.interpolate import interp1d
import sklearn

import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import sys

from torch.distributions import Independent, Distribution
from fab_priors import BoxUniform_New

import torch
from sbi.inference import SNRE_A, SNRE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi.types import Array, OneOrMore, ScalarFloat

from Coverage_eval_SBI import Coverage_class

np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=10_000)

In [2]:
op= '/home/mvasist/petitRADTRANS/petitRADTRANS/retrieval_examples/emission/'
observation_files = {}
observation_files['NIRISS SOSS'] = op +'NIRISS_SOSS_flux.dat'
observation_files['NIRSpec G395M'] = op +'NIRSpec_G395M_flux.dat'
observation_files['MIRI LRS'] = op +'MIRI_LRS_flux.dat'

# Wavelength range of observations, fixed parameters that will not be retrieved
WLENGTH = [0.3, 15.0]
# LOG_G =  2.58
R_pl =   1.84*nc.r_jup_mean
R_star = 1.81*nc.r_sun
gamma = 1
t_equ= 0

# Get host star spectrum to calculate F_pl / F_star later.
T_star = 6295.
x = nc.get_PHOENIX_spec(T_star)
fstar = interp1d(x[:,0], x[:,1])

In [3]:
####################################################################################
####################################################################################
### READ IN OBSERVATION
####################################################################################
####################################################################################

# Read in data, convert all to cgs! 

'''
Im using only data_flux_nu_error['MIRI LRS'] from here to calculate the likelihood. 
'''

data_wlen = {}
data_flux_nu = {}
data_flux_nu_error = {}
data_wlen_bins = {}

for name in observation_files.keys():
    print(name)
    dat_obs = np.genfromtxt(observation_files[name])
    data_wlen[name] = dat_obs[:,0]*1e-4
    data_flux_nu[name] = dat_obs[:,1]
    data_flux_nu_error[name] = dat_obs[:,2]
    
    data_wlen_bins[name] = np.zeros_like(data_wlen[name])
    data_wlen_bins[name][:-1] = np.diff(data_wlen[name])
    data_wlen_bins[name][-1] = data_wlen_bins[name][-2]

NIRISS SOSS
NIRSpec G395M
MIRI LRS


In [4]:
def Simulator(params): 

    NaN_spectra = 0

    atmosphere = Radtrans(line_species = ['H2O', 'CO_all_iso', \
                                         'CO2', 'CH4', \
                                          'Na', 'K'], \
          rayleigh_species = ['H2', 'He'], \
          continuum_opacities = ['H2-H2', 'H2-He'], \
          wlen_bords_micron = WLENGTH)#, mode='c-k')


    pressures = np.logspace(-6, 2, 100)
    atmosphere.setup_opa_structure(pressures)
    temperature = 1200. * np.ones_like(pressures)

    
    t_int = params[0].numpy()                             #200.
    log_kappa_IR = params[1].numpy()                      #-2
    log_gravity = params[2].numpy()                       #params[5].numpy() 1e1**2.45 

    gravity = np.exp(log_gravity)
    kappa_IR = np.exp(log_kappa_IR)
    
    temperature = nc.guillot_global(pressures, kappa_IR, gamma, gravity, t_int, t_equ)
    
    abundances = {}
    abundances['H2'] = 0.75 * np.ones_like(temperature) #0.74 * np.ones_like(temperature) (params[3].numpy())
    abundances['He'] = 0.25 * np.ones_like(temperature)  #0.24 * np.ones_like(temperature) (params[4].numpy())
    abundances['H2O'] = 0.001 * np.ones_like(temperature)
    abundances['CO_all_iso'] = 0.01 * np.ones_like(temperature)
    abundances['CO2'] = 0.00001 * np.ones_like(temperature)
    abundances['CH4'] = 0.000001 * np.ones_like(temperature)
    abundances['Na'] = 0.00001 * np.ones_like(temperature)
    abundances['K'] = 0.000001 * np.ones_like(temperature)

    MMW = rm.calc_MMW(abundances) * np.ones_like(temperature)
    #print(MMW, abundances)
    
    atmosphere.calc_flux(temperature, abundances, gravity, MMW)

    wlen, flux_nu = nc.c/atmosphere.freq, atmosphere.flux/1e-6


    # Just to make sure that a long chain does not die
    # unexpectedly:
    # Return -inf if forward model returns NaN values
    if np.sum(np.isnan(flux_nu)) > 0:
        print("NaN spectrum encountered")
        NaN_spectra += 1
        return torch.ones([1,371])* -np.inf

    # Convert to observation for emission case
    flux_star = fstar(wlen)
    flux_sq   = flux_nu/flux_star*(R_pl/R_star)**2 

    flux_rebinned = rgw.rebin_give_width(wlen, flux_sq, \
                data_wlen['MIRI LRS'], data_wlen_bins['MIRI LRS'])

    #flux_rebinned = np.reshape(flux_rebinned, (371,1))    

    FR= torch.Tensor(flux_rebinned)
    
    return FR    


In [5]:
Prior= utils.BoxUniform(low=torch.tensor([0., -4 , 2 ]), high=torch.tensor([2000., 0, 3.7 ]))

In [6]:
simulator, prior = prepare_for_sbi(Simulator, Prior)


  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
 Done.


  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
 Done.



In [7]:
simulator, prior = prepare_for_sbi(Simulator, Prior)


  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
 Done.


  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
 Done.



In [8]:
inference = SNRE_A(prior= Prior, device= 'cpu', classifier='mlp')

In [9]:
#reading the csv file

# Xpd= pd.read_csv('X_1M.csv')
# Tpd= pd.read_csv('T_1M.csv')

X=[]
T=[]

# for i in range(1,11):
# #     print(i)
#     dfX= pd.read_csv('/home/mvasist/simulations/3_params/X_'+ str(i) + '.csv')
#     dfT= pd.read_csv('/home/mvasist/simulations/3_params/T_'+ str(i) + '.csv')
#     X.append(dfX.values)
#     T.append(dfT.values)
    
# for j in range(1,51):
#     dfX_= pd.read_csv('/home/mvasist/simulations/3_params/_X_'+ str(i) + '.csv')
#     dfT_= pd.read_csv('/home/mvasist/simulations/3_params/_T_'+ str(i) + '.csv')
#     X.append(dfX_.values)
#     T.append(dfT_.values)
    
# for k in range(1, 101):
#     dfX_M= pd.read_csv('/home/mvasist/simulations/3_params/X_1Msim_'+ str(k) + '.csv', low_memory = False) #chunksize=1000, iterator=True, dtype={'col2': np.float32}
#     dfT_M= pd.read_csv('/home/mvasist/simulations/3_params/T_1Msim_'+ str(k) + '.csv', low_memory = False)
#     X.append(dfX_M.values) #for x,t in zip(dfX_M,dfT_M):
#     T.append(dfT_M.values)


#log gamma, Tequ and log gravity 
for k in range(1, 101):
    if k==15: continue
    dfX_M= pd.read_csv('/home/mvasist/simulations/3_params/1/X_100ksim_TintLkIRLg'+ str(k) + '.csv', low_memory = False) #chunksize=1000, iterator=True, dtype={'col2': np.float32}
    dfT_M= pd.read_csv('/home/mvasist/simulations/3_params/1/T_100ksim_TintLkIRLg'+ str(k) + '.csv', low_memory = False)
    X.append(dfX_M.values) #for x,t in zip(dfX_M,dfT_M):
    T.append(dfT_M.values)


In [10]:
# data = pd.read_csv(filepath, chunksize=1000, iterator=True) 
#Loop through the chunks and process the data
    
comb_np_array_X = np.vstack(X)
comb_np_array_T = np.vstack(T)

Xframe = pd.DataFrame(comb_np_array_X)
Tframe = pd.DataFrame(comb_np_array_T)

list_of_tensors_X = [torch.tensor(np.array(Xframe),dtype=torch.float32)]
list_of_tensors_T = [torch.tensor(np.array(Tframe),dtype=torch.float32)] #torch.tensor

XX = torch.cat(list_of_tensors_X)[:, 1:]
TT = torch.cat(list_of_tensors_T)[:,1:]



In [11]:
inference = inference.append_simulations(TT, XX) #TT[:1000], XX[:1000]

In [12]:
density_estimator = inference.train()

Neural network successfully converged after 114 epochs.


In [14]:
#i/p and o/p

for k in range(1, 101):
    if k==1: continue
    if k==15: continue
    dfX_M= pd.read_csv('/home/mvasist/simulations/3_params/1/X_10ksim_TintkIRLg'+ str(k) + '.csv', low_memory = False) #chunksize=1000, iterator=True, dtype={'col2': np.float32}
    dfT_M= pd.read_csv('/home/mvasist/simulations/3_params/1/T_10ksim_TintkIRLg'+ str(k) + '.csv', low_memory = False)
    X.append(dfX_M.values) #for x,t in zip(dfX_M,dfT_M):
    T.append(dfT_M.values)
    
comb_np_array_X = np.vstack(X)
comb_np_array_T = np.vstack(T)

Xframe = pd.DataFrame(comb_np_array_X)
Tframe = pd.DataFrame(comb_np_array_T)

list_of_tensors_X = [torch.tensor(np.array(Xframe),dtype=torch.float32)]
list_of_tensors_T = [torch.tensor(np.array(Tframe),dtype=torch.float32)] #torch.tensor

outputs = torch.cat(list_of_tensors_X)[:, 1:]
inputs = torch.cat(list_of_tensors_T)[:,1:]

In [14]:
# n = 10000
# indices = np.random.choice(np.arange(len(inputs)), n, replace=False)

# inputs_new = inputs[indices]
# outputs_new = outputs[indices]



In [ ]:

confidence_level = 0.95

ratio_estimator = inference

# cov = Coverage_class().coverage(ratio_estimator, inputs_new, outputs_new, confidence_level) # Play with the confidence level!
cov = Coverage_class().coverage(ratio_estimator, inputs, outputs, confidence_level) # Play with the confidence level!

cov

Coverages evaluated:  15%|█▍        | 17599/118305 [13:10<1:14:52, 22.42it/s]

In [ ]:
# inp = torch.tensor([ 2.3355e+02, -1.4847e-01,  2.4558e+00]) 
# out = torch.tensor([ 367.1692,  423.3806,  446.6609,  471.9345,  484.4444,  452.2699,
#          478.8342,  468.9159,  459.2518,  477.2024,  425.4767,  453.2717,
#          450.1346,  436.2054,  475.9433,  552.5604,  511.9610,  595.2401,
#          654.0010,  623.0190,  710.7034,  742.8564,  747.2242,  708.1611,
#          574.3649,  554.1912,  448.9724,  480.2288,  397.4876,  369.1758,
#          470.0436,  483.5425,  536.7816,  479.3451,  475.4352,  473.8654,
#          604.4164,  539.7837,  621.3248,  523.9114,  566.9241,  554.2429,
#          586.8416,  628.0115,  562.6287,  591.1508,  574.2996,  564.9891,
#          591.6976,  639.3524,  622.4237,  532.6631,  712.1290,  650.8790,
#          636.4476,  601.5356,  705.5764,  653.1938,  595.8697,  544.6038,
#          636.9045,  654.6161,  703.9503,  603.8680,  685.6127,  736.4333,
#          742.3319,  791.9485,  740.8298,  798.2189,  808.8278,  843.8052,
#          751.7430,  779.1505,  842.1805,  820.0933,  893.3881,  833.6739,
#          840.5039,  832.2475,  968.0964,  887.5250,  963.3968,  909.5132,
#          958.0134,  938.8223,  979.9813, 1032.8459, 1032.2262, 1016.5588,
#         1041.3066, 1022.2029, 1012.5118, 1085.5587, 1014.9733, 1102.5543,
#         1113.9183, 1148.6987, 1142.8439, 1089.6693, 1111.3379, 1137.0232,
#         1105.9293, 1177.5829, 1143.2031, 1159.9249, 1188.4000, 1152.5513,
#         1178.5624, 1194.1971, 1220.3744, 1179.5607, 1146.0399, 1238.0752,
#         1243.8077, 1296.1718, 1184.7209, 1181.9935, 1155.9821, 1260.4266,
#         1285.1157, 1273.3434, 1341.0625, 1358.8638, 1302.8070, 1255.6371,
#         1251.2518, 1268.9768, 1304.8105, 1297.1256, 1333.7682, 1367.6329,
#         1361.5414, 1384.6652, 1298.5760, 1344.9980, 1311.4451, 1250.4990,
#         1274.5532, 1374.8159, 1410.2544, 1386.5212, 1378.3256, 1381.2404,
#         1404.3063, 1347.5524, 1389.8744, 1404.7419, 1420.2759, 1444.0844,
#         1442.9592, 1491.9259, 1417.5511, 1443.3881, 1424.3118, 1368.3845,
#         1418.1633, 1396.5315, 1436.0254, 1425.0453, 1412.1172, 1268.4159,
#         1289.2551, 1454.8029, 1404.6243, 1407.4402, 1483.6171, 1502.0938,
#         1456.6106, 1347.9098, 1346.9360, 1411.4769, 1422.6282, 1433.3041,
#         1463.6768, 1500.3660, 1263.6820, 1441.2551, 1384.3868, 1430.0222,
#         1491.7272, 1573.2211, 1469.4840, 1456.3807, 1426.3516, 1415.6198,
#         1498.7864, 1303.7125, 1419.7417, 1350.1771, 1224.8715, 1403.5442,
#         1319.6971, 1314.2812, 1255.0343, 1422.6686, 1505.2051, 1485.2242,
#         1474.9879, 1458.3916, 1425.0537, 1449.0519, 1450.6737, 1468.5631,
#         1474.7427, 1471.7065, 1531.3967, 1438.2906, 1354.3418, 1542.8225,
#         1507.8424, 1471.9069, 1540.9037, 1605.0164, 1465.6986, 1404.2490,
#         1448.0466, 1518.5066, 1592.2638, 1498.7589, 1358.5913, 1328.1338,
#         1518.7521, 1477.4407, 1376.2795, 1397.3727, 1415.7776, 1523.0573,
#         1476.7186, 1465.8645, 1205.7474, 1223.7684, 1222.9697, 1379.4788,
#         1507.3293, 1465.8646, 1358.6455, 1529.2432, 1471.0265, 1481.2397,
#         1395.8068, 1419.3809, 1529.0793, 1494.0096, 1347.0781, 1553.3522,
#         1472.1287, 1519.6536, 1490.8920, 1466.7394, 1423.0494, 1333.2438,
#         1456.8129, 1504.0338, 1473.9674, 1415.2137, 1562.8751, 1606.4968,
#         1513.1086, 1530.9871, 1503.1689, 1380.5745, 1348.1449, 1269.9005,
#         1526.8435, 1482.1417, 1508.5475, 1526.8644, 1451.3569, 1396.1167,
#         1262.3923, 1373.9795, 1408.4009, 1248.5408, 1353.6376, 1466.9718,
#         1421.9869, 1354.8962, 1252.2578, 1370.1368, 1364.4688, 1389.8683,
#         1262.6771, 1304.8265, 1358.2997, 1344.1091, 1387.2651, 1391.8475,
#         1342.4696, 1440.7512, 1477.5764, 1376.4623, 1351.4304, 1258.3092,
#         1350.5327, 1443.3967, 1358.9387, 1358.1565, 1345.9637, 1307.9398,
#         1381.8298, 1394.6443, 1418.5308, 1387.3458, 1399.5120, 1392.7764,
#         1379.8488, 1415.8597, 1325.0372, 1353.6196, 1363.4506, 1169.9156,
#         1182.3392, 1350.6189, 1394.2982, 1373.4652, 1448.2311, 1423.0044,
#         1238.5383, 1270.9479, 1347.1368, 1382.1384, 1399.9271, 1305.4482,
#         1204.6395, 1210.8241, 1185.0779, 1152.2759, 1226.5284, 1170.2477,
#         1198.1663, 1296.7449, 1271.6583, 1363.6239, 1356.8755, 1423.4586,
#         1403.0626, 1360.2222, 1281.9836, 1174.9844, 1210.4607, 1166.8698,
#         1161.2554, 1032.0295, 1056.6460, 1098.6243, 1276.1512, 1254.6946,
#         1198.9166, 1325.6124, 1275.3268, 1267.9679, 1192.1652, 1296.3091,
#         1301.8081, 1279.4624, 1274.4971, 1253.7152, 1122.0195, 1187.8660,
#         1152.3267, 1216.2982, 1349.2916, 1253.1123, 1183.8849, 1276.3474,
#         1307.2947, 1237.3458, 1236.3699, 1228.8662, 1121.4292])

# inp_, out_ = inp.squeeze().unsqueeze(0), out.squeeze().unsqueeze(0)
# inp_.size(), out_.size
# inp_r= inp_.repeat(2,1)
# out_r= out_.repeat(2,1)
# num_atoms=2
# batch_size = inp_r.shape[0]
# print(batch_size)
# repeated_x = utils.repeat_rows(out_r, num_atoms)

# # Choose `1` or `num_atoms - 1` thetas from the rest of the batch for each x.
# probs = torch.ones(batch_size, batch_size) * (1 - torch.eye(batch_size)) / (batch_size - 1)
# print(probs)

# choices = torch.multinomial(probs, num_samples=num_atoms - 1, replacement=False)

# contrasting_theta = inp_r[choices]

# atomic_theta = torch.cat((inp_r[:, None, :], contrasting_theta), dim=1).reshape(
#     batch_size * num_atoms, -1
# )